#Load Data

In [286]:
import pandas as pd
data = pd.read_csv('DB SMAE V11_3.csv', sep= ';', encoding = 'iso-8859-1', na_values=[' ','',str,'ND','#¿NOMBRE?'], header=0)

In [ ]:
data = data.fillna(0)
target_txt = data.Subgrupo.unique().tolist()
target_txt.sort()
target_map = { sg: i for i, sg in enumerate(target_txt)}


target = data.Subgrupo.map(target_map).values
data['target'] = target
data.columns

df = data[[ 'Energía (kcal/100 g)', 'Proteína (g/100 g)',
       'Lípidos totales (g/100 g)', 'AG Saturados (g/100 g)',
       'Colesterol (g/100 g)', 'Hidratos de Carbono ((g/100 g)',
       'Azúcar (g/100 g)', 'Fibra (g/100 g)', 'Na, Sodio [mg/100 g]',
       'target']]

#Random Forest

In [288]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(['target'], axis = 'columns'), target, test_size = 0.2)


In [329]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train )
model.score(X_test, y_test)

0.8298611111111112

#MLP

In [332]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

X_train, X_test, y_train, y_test = train_test_split(df.drop(['target'], axis = 'columns'), target, test_size = 0.2)
mlp = MLPClassifier(hidden_layer_sizes = (100,60), activation = 'logistic', learning_rate = 'adaptive')
mlp
mlp.fit(X_train, y_train )


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 60),
              learning_rate='adaptive')

In [333]:
mlp.score(X_test, y_test)


0.8454861111111112

#XGBoost

In [ ]:
import xgboost
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(df.drop(['target'], axis = 'columns'), target, test_size = 0.5, random_state = 1)
# Convert feature names to strings if needed
X_train.columns = X_train.columns.astype(str)
# Replace problematic characters in feature names
X_train.columns = X_train.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_')

X_test.columns = X_test.columns.astype(str)
# Replace problematic characters in feature names
X_test.columns = X_test.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_')

X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size = 0.5,  random_state = 1)

xgb = xgboost.XGBClassifier()

parameters = {
                'nthreads': [4]
              ,'objective': ['multi:softprob', 'multi:softmax']
            #   ,'num_class': [19]
            #   ,'early_stopping_rounds': [10]
                ,'max_depth' : [9,6]
              ,'learning_rate': [0.15]
              ,'n_estimators': [3000]}
fit_param = {
                'early_stopping_rounds': 10
              ,'eval_metric': 'mlogloss'
              ,'eval_set':[(X_test, y_test)]}

clf = GridSearchCV(xgb, parameters, cv=4, scoring = 'accuracy')
clf.fit(X_train, y_train, **fit_param)

In [273]:
clf.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, nthreads=4, num_parallel_tree=None,
              objective='multi:softprob', ...)

In [278]:
clf.best_score_

0.8025050294026617

In [275]:
from sklearn.metrics import accuracy_score
best_xgb = clf.best_estimator_
y_preds = best_xgb.predict(X_valid)

comp = pd.DataFrame({'real':y_valid, 'Preds':y_preds })
comp

,real,Preds
0,8,8
1,17,17
2,8,8
3,7,7
4,17,17
...,...,...
714,3,1
715,16,18
716,5,8
717,11,7


In [276]:
acc = accuracy_score(y_valid, y_preds)
acc

0.8136300417246175